# Uncertainties in python (simple examples)

(from the Example in the guides prepared by Dirk)

In [1]:
from uncertainties import ufloat, ufloat_fromstr
from uncertainties.umath import *  # sin(), etc.
from uncertainties import unumpy
import numpy as np

## Example: Ratio of quantities

Determine $R=\frac{Y(511\,\text{keV})}{2Y(1275\,\text{keV})}$


In [2]:
# imput data
# Y1 -> Y(511keV)
# Y2 -> Y(1275keV)
Y1 = [ ufloat(203500, sqrt(203500)), # <-- basically Y +/- sqrt(Y)
       ufloat(220100, sqrt(220100)), 
       ufloat(157400, sqrt(157400)), 
       ufloat(188300, sqrt(188300))]
Y2 = [ ufloat(115500, sqrt(115500)), 
       ufloat(121100, sqrt(121100)), 
       ufloat( 89100, sqrt( 89100)), 
       ufloat(107200, sqrt(107200)) ]

# turn the python lists into numpy arrays
Y1 = np.array(Y1)
Y2 = np.array(Y2)

In [3]:
def avg_simple(to_be_avg):
    """
    simple average: add the values and divided by the number of values.
    
    This intends to introduce the benefit of using numpy arrays instead 
    of coding your own functions always. Sometimes one has to anyway.
    """
    avg = 0
    for val in to_be_avg:
        avg += val
    return avg/len(to_be_avg)

print("Simple average (function): {:.2uS}".format(avg_simple(Y1/(2*Y2))))

Simple average (function): 0.8878(17)


## Weighted average vs. simple average

As we saw from before, the average is a simple mathematical operation that it does not take into account the unceirtainties of each individual quantity

$$
A = \frac{\sum_{i=1}^n X_i}{n}
$$

the better way to average quantities with unceirtainties is to *weight* each value to be averaged by its weight.

$$
W_A = \frac{\sum_{i=1}^n w_i X_i}{\sum_{i=1}^n w_i}
$$

let's see the difference with a numerical example

In [6]:
# 53mCo proton emission energies in keV measured with different 
# setups and experiments ober the years 
jac1970 = ufloat_fromstr("1560(40)")
cer1972 = ufloat_fromstr("1590(30)")
she2015 = ufloat_fromstr("1558(8)")
kan2010 = ufloat_fromstr("1559(7)")
nes2017 = ufloat_fromstr("1558.8(17)")

# create the numpy array to be averaged
to_be_avg = [jac1970, cer1972, she2015, kan2010, nes2017]

def avg_weight(to_be_avg):
    """
    weighted average: add the values and divided by the number of values.
    """
    num = 0
    den = 0
    for val in to_be_avg:
        num += val.n / val.s**2
        den +=     1 / val.s**2
        
    mean = num/den
    sigma = np.sqrt( 1 / den )
    avg_weight = ufloat( mean, sigma )
    return avg_weight

#print("WA = {:S} keV (weighted)".format(avg_weight(to_be_avg)))
#print(" A = {:S} keV (simple)".format(avg_simple(to_be_avg )))

inside = [ufloat(0.01, 0.001), 
          ufloat(0.01, 0.001), 
          ufloat(0.04, 0.004), 
          ufloat(0.1, 0.01),
          ufloat(0.01, 0.001)]

outside = [ufloat(0.01, 0.001), 
          ufloat(0.01, 0.001), 
          ufloat(0.01, 0.001), 
          ufloat(0.05, 0.005),
          ufloat(0.02, 0.002)]

print(f'inside = {avg_weight(inside)} uSv/h and {avg_weight(inside) * 8765.81277} uSv/yr')
print(f'outside = {avg_weight(outside)} uSv/h and {avg_weight(outside) * 8765.81277} uSv/yr')


inside = 0.0109+/-0.0006 uSv/h and 96+/-5 uSv/yr
outside = 0.0112+/-0.0006 uSv/h and 99+/-5 uSv/yr


## Using numpy directly to get averages

One can always use `numpy` directly to get those averages. The only drawback is that I do not know how to get the propagation of the uncertainty properly/easily done. 


In [ ]:
print()
print("We can get numpy to do it for us")
values_to_be_avg  = unumpy.nominal_values(to_be_avg) # <-- get the values only
weigths_to_be_avg =1/unumpy.std_devs(to_be_avg)**2
print("weighted average (numpy): {:.5} keV".format(np.average(values_to_be_avg, weights=weigths_to_be_avg)))
print("         average (numpy): {:.5} keV".format(np.average(unumpy.nominal_values(values_to_be_avg))))
print()
print("HOWEVER, I (Pico) currently do not know an easy way to get the weighted uncertainty")

# Example of an energy calibration

Here we also show another way to use values with unceirtanties

In [ ]:
from uncertainties import ufloat_fromstr
# Note, no errors reported in the y axis.
# BUT we always have the systematic one from the precision. 0.1 in this case.
y_i = [
    ufloat_fromstr("279.2"),
    ufloat_fromstr("511.0"),
    ufloat_fromstr("1274.5"),
    ufloat_fromstr("2614.5"),
]
# errors in the x axis from the uncertainty of the fit in the centroid of the peak.
x_i = [
    ufloat_fromstr("107.1(5)"),
    ufloat_fromstr("180.1(7)"),
    ufloat_fromstr("421.3(11)"),
    ufloat_fromstr("841.9(18)"),
]

y  = [ i.n for i in y_i ] # n -> nominal value
dy = [ i.s for i in y_i ] # s -> standard deviation.

x  = [ i.n for i in x_i ] # n -> nominal value
dx = [ i.s for i in x_i ] # s -> standard deviation

In [ ]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

fig = plt.figure()
fig.suptitle("Energy calibration", fontsize=14, fontweight='bold')

ax = fig.add_subplot(111)
fig.subplots_adjust(top=0.85)

ax.set_xlabel("channel [a.u.]")
ax.set_ylabel("Energy [keV]")

# plot dots
plt.scatter(x, y)
#plot with error bars. OBS: They are smaller than the scatter dot.
plt.errorbar(x, y, xerr=dx, yerr=dy, linestyle=':' )

plt.show()

# Let's so some fitting

## Attempt 1: (WRONG ONE) No errors included

In [ ]:
from scipy import stats
import numpy as np
b, a, r_value, p_value, std_err = stats.linregress(x,y)
print(" E = {:.1f}*ch + {:.1f}".format(b,a))
## not a clue on how to estimate the errors =(
## I think std_err it is the error of the slope only, error for offset?

## Attempt 2: (STILL WRONG) Errors only on the $y$ axis

In [ ]:
import numpy as np
from scipy import optimize

def f(x, a, b):
    "a line"
    return a + b*x

def ff(x, p):
    "helper function"
    return f(x, *p)

# initial guesses
a = 0
b = 1

pstart = [a,b]

def fit_curvefit(p0, datax, datay, function, yerr=None, **kwargs):
    "Generic fitting function for when one has errors or not in the input data"

    if yerr is not None:
        pfit, pcov = \
             optimize.curve_fit(f,datax,datay,p0=p0,\
                                sigma=yerr, epsfcn=0.0001, **kwargs)
    else:
        pfit, pcov = \
             optimize.curve_fit(f,datax,datay,p0=p0,\
                                epsfcn=0.0001, **kwargs)
    error = [] 
    for i in range(len(pfit)):
        try:
          error.append(np.absolute(pcov[i][i])**0.5)
        except:
          error.append( 0.00 )
    pfit_curvefit = pfit
    perr_curvefit = np.array(error)
    return pfit_curvefit, perr_curvefit 

##
## Do the fit with the values only
##
pfit, perr = fit_curvefit(pstart, x, y, ff)

print("\n# Fit parameters and parameter errors from curve_fit method :")
print("pfit = ", pfit)
print("perr = ", perr)
a = ufloat(pfit[0],perr[0])
b = ufloat(pfit[1],perr[1])
print("a = {:S}".format(a))
print("b = {:S}".format(b))

ch = 227.3
E = a + ch*b
print("E @ {:.1f} = {:S}".format(ch, E))

##
## Do the fit with the uncertainty only on the y axis
##
pfit, perr = fit_curvefit(pstart, x, y, ff, dy)

print("\n# Fit parameters and parameter errors from curve_fit method :")
print("pfit = ", pfit)
print("perr = ", perr)
a = ufloat(pfit[0],perr[0])
b = ufloat(pfit[1],perr[1])
print("a = {:S}".format(a))
print("b = {:S}".format(b))

E = a + ch*b
print("E @ {:.1f} = {:S}".format(ch,E))


### Results with y-errors included
# Fit parameters and parameter errors from curve_fit method :
#pfit =  [-61.96861944   3.17793761]
#perr =  [1.65069473 0.00342302]
#a = -62.0(1.7)
#b = 3.1779(34)
#E = 660.4(1.8)

## The unceirtainties are on x...
#the results including unceirtanties in y are not suprisingly
# basically IDENTICAL


## Attempt 3: (Now we are talking) error on both axes 

Unlike the previous example, we do not use the curve_fit module of Scipy, Instead, there is  another  dedicated module to estimate the orthogonal distance regression (odr). 

**Note that the uncertainties in the y axis are assumed to be the systematic ones only**

The program with some comments is shown below:

In [ ]:
import numpy as np
from pylab import *
from scipy.optimize import curve_fit
from scipy import odr

def func(p, x):
    "function to fit"
    a, b = p
    return a + b*x

# Model object
quad_model = odr.Model(func)

# Create a RealData object
data = odr.RealData(x, y, sx=dx, sy=dy)

# Set up ODR with the model and data.
odr = odr.ODR(data, quad_model, beta0=[0., 1.]) # initial guess of parameters

# Run the regression.
out = odr.run()

#print fit parameters and 1-sigma estimates
popt = out.beta
perr = out.sd_beta
print('fit parameter 1-sigma error')
print('———————————–')
for i in range(len(popt)):
    print("{:6.2f} +/- {:6.2f}".format(popt[i], perr[i]))
print()
    
a = ufloat(popt[0],perr[0])
b = ufloat(popt[1],perr[1])
print("a = {:uS}".format(a))
print("b = {:uS}".format(b))

ch = ufloat_fromstr("227.3(8)")
E = a + ch*b
print("E = {:uS}".format(E))

# prepare confidence level curves
nstd = 5. # to draw 5-sigma intervals
popt_up = popt + nstd * perr
popt_dw = popt - nstd * perr

x_fit = np.linspace(min(x), max(x), 100)
fit = func(popt, x_fit)
fit_up = func(popt_up, x_fit)
fit_dw = func(popt_dw, x_fit)

#plot
fig, ax = plt.subplots(1)
rcParams['font.size']= 20
xlabel('x', fontsize=16)
ylabel('y', fontsize=16)
title('fit with error on both axis', fontsize=18)
## plot points with errors in both axes
errorbar(x, y, yerr=dy, xerr=dx, ecolor='k', fmt='none', label='data')
## plot line corresponding to fit
plot(x_fit, fit, 'r', lw=0.5, label='best fit curve')
## color a 5 sigma region to the fit 
ax.fill_between(x_fit, fit_up, fit_dw, alpha=.25, label='5-sigma interval')
legend(loc='lower right',fontsize=16)
show()
## OBS: 5-sigma thickness is comparable/smaller with best fit sigma